# OpenHPC環境の削除

---

構築したOpenHPC環境を削除します。

## パラメータの指定

削除を行うのに必要となるパラメータを入力します。

### VCCアクセストークンの入力

VCノード, VCディスクを削除するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### グループ名

OpenHPCのUnitGroup名を指定してください。

> VCノードを起動した際に指定したものと同じ名前を指定してください。

In [ ]:
ugroup_name = 'OpenHPC'

## 構築環境の削除

構築したOpenHPC環境を削除します。

### VCノードの削除

起動したVCノードを削除します。

現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

現在のVCノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

まず計算ノード用VCノードを削除します。

In [ ]:
ug.delete_units('compute', force=True)

マスターノードと UnitGroup の削除を行います。

In [ ]:
ug.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### VCディスクの削除

NFS用のVCディスクを削除します。

> VCディスクを作成していない場合は、何もしません。

現在の状態を確認します。

In [ ]:
from IPython.display import display
ug_disk = vcp.get_ugroup(ugroup_name + '_disk')
if ug_disk:
    display(ug_disk.df_nodes())

VCディスクを削除します。

In [ ]:
if ug_disk:
    ug_disk.cleanup()

削除後のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

## Ansible設定のクリア

削除した環境に対応するAnsibleの設定をクリアします。

### group_varsファイル

group_varsファイルをリネームします。

In [ ]:
!mv group_vars/{ugroup_name}.yml group_vars/{ugroup_name}.yml.bak

### インベントリ

インベントリから UnitGroup に対応するグループを削除します。

In [ ]:
import yaml
from pathlib import Path

inventory_path = Path('inventory.yml')

!cp {str(inventory_path)} {str(inventory_path)}.bak

with inventory_path.open() as f:
    inventory = yaml.safe_load(f)
    
if ugroup_name in inventory['all']['children']:
    del(inventory['all']['children'][ugroup_name])

with inventory_path.open(mode='w') as f:
    f.write(yaml.safe_dump(inventory))
    
!cat {str(inventory_path)}